In [1]:
import pandas as pd
import numpy as np
import h5py

In [2]:
samples = pd.read_csv('Reviews.csv')

In [4]:
samples['cleaned'] = samples['Review'].str.lower()
samples['cleaned'] = samples['cleaned'].str.replace(r'[^a-z\s\']', ' ', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'^\s+', '', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'\s+$', '', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'\'', '', regex=True)
samples['cleaned'] = samples['cleaned'].str.replace(r'\s+', ' ', regex=True)

samples['cleaned']

0        working with one of the best shakespeare sourc...
1        well tremors i the original started off in and...
2        ouch this one was a bit painful to sit through...
3        ive seen some crappy movies in my life but thi...
4        carriers follows the exploits of two guys and ...
                               ...                        
49995    this movie is certainly well constructed begin...
49996    nice to see a comedy for grown ups masterfully...
49997    jean renoirs homage to the paris of the late t...
49998    what are the movies i mean what are movies mad...
49999    i saw this movie on tv and loved it i am a rea...
Name: cleaned, Length: 50000, dtype: object